# Load the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats

## Load the data and preprocess it

In [ ]:
# ===========================================
# Interannual variability of Nino 3.4, NAO, and CHL anomalies (1998–2024)
# 3 subplots (columns): Nino 3.4, NAO, CHL (detrended anomaly)
# ===========================================


# -----------------------------
# Load dataset
# -----------------------------
df = pd.read_csv("merged_data_with_indexes.csv", parse_dates=['time'])
df = df.dropna(subset=['time'])
df = df.sort_values('time')

# -----------------------------
# CHL anomaly computation
# -----------------------------
chl_df = df[['time', 'Chlorophyll-a']].dropna()
chl_df['month'] = chl_df['time'].dt.month

# Compute monthly climatology and anomalies
chl_climatology = chl_df.groupby('month')['Chlorophyll-a'].mean()
chl_df['chl_anomaly'] = chl_df.apply(lambda x: x['Chlorophyll-a'] - chl_climatology.loc[x['month']], axis=1)

# Detrend CHL anomaly
t = np.arange(len(chl_df))
slope, intercept, *_ = stats.linregress(t, chl_df['chl_anomaly'])
chl_df['chl_detrended'] = chl_df['chl_anomaly'] - (slope * t + intercept)

# -----------------------------
# Prepare time series (already detrended)
# -----------------------------
time = df['time']
nino34 = df['Nino34']                 # Already detrended
nao = df['NorthAtlanticOscillation']  # Already detrended

# -----------------------------
# Plot setup (1 row × 3 columns)
# -----------------------------
fig, axes = plt.subplots(3, 1, figsize=(14, 9), sharex=True, constrained_layout=True)

colors = {
    "Nino34":"#ff260e",  # blue
    "NAO": "#4f0906",     # red
    "CHL": "#2ca02c"      # green
}


# variables_colors = {
    # 'SST_cv':       {'pos': "#ff260e", 'neg': "#68b4ea"},   
#     'CHL_cv':       {'pos': '#2ca02c', 'neg': "#b2c09c"},
#     'AOD_cv':       {'pos': "#4f0906", 'neg': "#393ed6bf"},
#     'Wind_Speed_cv':{'pos': "#ca2b0b", 'neg': "#3d7bf0"}
# }
# Common plot limits
x_min, x_max = pd.to_datetime('1998-01-01'), pd.to_datetime('2024-12-31')

# 1️⃣ Nino 3.4
ax = axes[0]
ax.axhline(0, color='k', linestyle='--', alpha=0.6)
ax.fill_between(time, nino34, where=nino34 >= 0, color=colors["Nino34"], alpha=0.7)
ax.fill_between(time, nino34, where=nino34 < 0, color="#68b4ea", alpha=0.7)
ax.set_title("a)", fontsize=14, fontweight='bold', fontfamily='Times New Roman', loc='left')
ax.grid(alpha=0.3)
ax.set_xlim(x_min, x_max)

# 2️⃣ NAO
ax = axes[1]
ax.axhline(0, color='k', linestyle='--', alpha=0.6)
ax.fill_between(time, nao, where=nao >= 0, color=colors["NAO"], alpha=0.7)
ax.fill_between(time, nao, where=nao < 0, color="#393ed6bf", alpha=0.7)
ax.set_title("b)", fontsize=14, fontweight='bold', fontfamily='Times New Roman', loc='left')
ax.grid(alpha=0.3)
ax.set_xlim(x_min, x_max)

# 3️⃣ CHL (detrended anomaly)
ax = axes[2]
ax.axhline(0, color='k', linestyle='--', alpha=0.6)
ax.fill_between(chl_df['time'], chl_df['chl_detrended'], where=chl_df['chl_detrended'] >= 0,
                color=colors["CHL"], alpha=0.7)
ax.fill_between(chl_df['time'], chl_df['chl_detrended'], where=chl_df['chl_detrended'] < 0,
                color="#b2c09c", alpha=0.7)
ax.set_title("c)", fontsize=14, fontweight='bold', fontfamily='Times New Roman', loc='left')
ax.grid(alpha=0.3)
ax.set_xlim(x_min, x_max)

# Common X-axis formatting
axes[-1].xaxis.set_major_locator(mdates.YearLocator(2))
axes[-1].xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
for ax in axes:
    ax.tick_params(axis='x', rotation=90)

# Global label
fig.supxlabel("Year", fontsize=14, fontfamily='Times New Roman')

# -----------------------------
# Save and show
# -----------------------------
plt.savefig('interannual_anomalies_three_subplots_with_titles_Updated.png', dpi=360, bbox_inches='tight', facecolor='white')
plt.show()
